#### Import libraries

In [43]:
# libraries
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
from torchvision.transforms import transforms
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
from datetime import datetime

# check PyTorch version
print(f"PyTorch version: {torch.__version__}")

# check device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

# created date
print("Created date: 2023:08:07 18:17:12")

# modified date
now = datetime.now().strftime("%Y:%d:%m %H:%M:%S")
print(f"Modified date: {now}")


PyTorch version: 1.12.1+cu102
Device: cuda
Created date: 2023:08:07 18:17:12
Modified date: 2023:08:07 23:52:02


#### Dataset

1. Calculate `mean` and `standard deviation(std)` from training set

In [35]:
# download training set
trainset = FashionMNIST(root="../../data", download= True, train=True, transform=transforms.Compose([transforms.ToTensor()]))

# load all data from training set
trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=True)

# train_data
data = next(iter(trainloader))

# check length
print(f"train_data size: {len(data[0])}") # data[0]: train_data, data[1]: train_labels

# calculate mean, std
mean = data[0].mean()
std = data[0].std()

print(f"mean: {mean} | std: {std}")

train_data size: 60000
mean: 0.2860405743122101 | std: 0.3530242443084717


2. Normalize training set, testing set with `mean` and `std`

In [36]:
# normalized_transform
normalized_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(std=std, mean=mean)])

# training set
trainset = FashionMNIST(root="../../data", download= True, train=True, transform= normalized_transform)

# testing set
testset = FashionMNIST(root="../../data", download= True, train=False, transform= normalized_transform)


#### Modeling

1. Create a `FashionMNISTNeuralNetwork` class for training

In [37]:
class FashionMNISTNeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=784, out_features= 256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features= 128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features= 64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features= 10),
        )

    def forward(self, x):
        return self.layers(x)

2. Implement training with a loop

In [42]:
# set manual seed
torch.manual_seed(42)

# num of epochs
epochs = 50

# batch size
batch_size = 16

# dataloader
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False)

# create an instance of the model
fashionmnist_model = FashionMNISTNeuralNetwork()

# loss function
loss_fn = nn.CrossEntropyLoss()

# optimizer
optimizer = torch.optim.Adam(params=fashionmnist_model.parameters(), lr=0.001)

# tranfer to device
fashionmnist_model.to(device)

FashionMNISTNeuralNetwork(
  (layers): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=256, bias=True)
    (2): ReLU()
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): ReLU()
    (5): Linear(in_features=128, out_features=64, bias=True)
    (6): ReLU()
    (7): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [44]:
# init weights 
for module in fashionmnist_model.modules():
    if isinstance(module, nn.Linear):
        nn.init.kaiming_normal_(module.weight)
        nn.init.constant_(module.bias, 0.0)

In [45]:
# training loop
for epoch in range(epochs):

    ### TRAINING 
    accumulated_train_batches = 0
    accumulated_train_losses = 0.0
    accumulated_train_accuracy = 0.0

    for inputs, targets in trainloader:

        # copy data to device
        inputs = inputs.to(device)
        targets = targets.to(device)

        # train mode
        fashionmnist_model.train()

        # forward pass
        y_logits = fashionmnist_model(inputs).squeeze()

        # calculate loss
        loss = loss_fn(y_logits, targets)

        # accumulate losses
        accumulated_train_losses += loss

        # accumulate batches
        accumulated_train_batches += 1

        # calculate accuracy
        acc = torch.eq(targets, torch.argmax(torch.softmax(y_logits.data, dim=1), dim=1)).sum().item()
        accumulated_train_accuracy += acc

        # zero gradients
        optimizer.zero_grad()

        # backward 
        loss.backward()

        # update parameters
        optimizer.step()

    train_loss = accumulated_train_losses / accumulated_train_batches
    train_acc = accumulated_train_accuracy / (accumulated_train_batches * batch_size)

    ### TESTING
    fashionmnist_model.eval()
    with torch.inference_mode():

        accumulated_test_batches = 0
        accumulated_test_losses = 0.0
        accumulated_test_accuracy = 0.0

        for inputs, targets in testloader:

            # copy data to device
            inputs = inputs.to(device)
            targets = targets.to(device)

            # forward pass
            y_logits = fashionmnist_model(inputs).squeeze()

            # calculate loss
            test_loss = loss_fn(y_logits, targets)

            # accumulate losses
            accumulated_test_losses += test_loss

            # accumulate batches
            accumulated_test_batches += 1

            # calculate accuracy
            acc = torch.eq(targets, torch.argmax(torch.softmax(y_logits.data, dim=1), dim=1)).sum().item()
            accumulated_test_accuracy += acc

    test_loss = accumulated_test_losses / accumulated_test_batches
    test_acc = accumulated_test_accuracy / (accumulated_test_batches * batch_size)


    print(f"Epoch: {epoch+1} / {epochs} | Train loss: {train_loss: 0.5f} | Train accuracy: {train_acc: 0.2f} | Test loss: {test_loss: 0.5f} | Test accuracy: {test_acc: 0.2f}")


Epoch: 1 / 50 | Train loss:  0.46943 | Train accuracy:  0.83 | Test loss:  0.41731 | Test accuracy:  0.84
Epoch: 2 / 50 | Train loss:  0.36495 | Train accuracy:  0.86 | Test loss:  0.38994 | Test accuracy:  0.86
Epoch: 3 / 50 | Train loss:  0.33088 | Train accuracy:  0.88 | Test loss:  0.36735 | Test accuracy:  0.86
Epoch: 4 / 50 | Train loss:  0.30686 | Train accuracy:  0.89 | Test loss:  0.35247 | Test accuracy:  0.87
Epoch: 5 / 50 | Train loss:  0.28952 | Train accuracy:  0.89 | Test loss:  0.38665 | Test accuracy:  0.87
Epoch: 6 / 50 | Train loss:  0.27533 | Train accuracy:  0.90 | Test loss:  0.37769 | Test accuracy:  0.87
Epoch: 7 / 50 | Train loss:  0.26378 | Train accuracy:  0.90 | Test loss:  0.33948 | Test accuracy:  0.88
Epoch: 8 / 50 | Train loss:  0.25415 | Train accuracy:  0.90 | Test loss:  0.35115 | Test accuracy:  0.88
Epoch: 9 / 50 | Train loss:  0.24289 | Train accuracy:  0.91 | Test loss:  0.34494 | Test accuracy:  0.88
Epoch: 10 / 50 | Train loss:  0.23255 | Train 